In [37]:
import os
import pandas as pd
from scipy.stats import ttest_ind
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool, Tool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.prompts import PromptTemplate

import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

# 假设我们有一个DataFrame
df = pd.DataFrame({
    '产品A_评分': [8, 7, 9, 8, 8, 7, 9, 10, 6, 7],
    '产品B_评分': [6, 7, 5, 6, 7, 5, 8, 6, 5, 6]
})

In [38]:
@tool
def t_test_tool(column_names: str) -> str:
    """
    当你需要比较数据集中两列数值的均值是否存在统计学上的显著差异时，使用此工具。
    该工具会执行一个双样本T检验。
    输入应该是一个字符串，包含两个用逗号分隔的列名，例如：产品A_评分,产品B_评分。
    """
    try:
        col1_name, col2_name = column_names.split(',')
        col1_data = df[col1_name.strip()]
        col2_data = df[col2_name.strip()]
        
        # 执行T检验
        t_stat, p_value = ttest_ind(col1_data, col2_data)
        
        # 根据p值判断结果
        if p_value < 0.05:
            return f"检验结果显著：T统计量为 {t_stat:.2f}, P值为 {p_value:.4f}。两组数据的均值存在显著差异。"
        else:
            return f"检验结果不显著：T统计量为 {t_stat:.2f}, P值为 {p_value:.4f}。两组数据的均值没有显著差异。"
            
    except Exception as e:
        return f"执行出错: {e}。请确保输入的列名正确且数据为数值型。"

In [42]:
@tool
def save_plot_tool(filename: str) -> str:
    """
    当你需要将刚刚生成的最新图表保存为本地图片文件时，使用此工具。
    请确保在调用此工具前，已经有一个图表被生成。
    输入是你想要保存的文件名，必须以.jpg, 或 .pdf结尾。例如: '产品销售额对比.jpg'。注意文件名的末尾不要有'\n'字符。
    """
    try:
        plt.savefig(filename)
        return f"图表已成功保存为 {filename}"
    except Exception as e:
        return f"保存图表时出错: {e}"

In [43]:
# 初始化LLM实例
api_key = os.environ['OPENAI_API_KEY']
llm = OpenAI(model='deepseek-chat', api_key=api_key, base_url="https://api.deepseek.com/beta")

# 创建Pandas Agent作为基础工具
pandas_agent_executor = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True
)

# 组装工具列表
# 将所有工具放入一个列表
tools = [t_test_tool, save_plot_tool, 
         Tool(
             name="Pandas Agent",
             func=pandas_agent_executor.invoke,
             description="""当你需要执行DataFrame的操作时，使用此工具。输入应该是关于该数据框的完整问题。"""
        )]

# 创建总管Agent
# 我们直接在代码中定义ReAct提示模板
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt_template = PromptTemplate.from_template(template)

# 创建总管Agent，它知道自己拥有哪些工具
main_agent = create_react_agent(llm, tools, prompt_template)

# 创建Agent执行器，这是驱动Agent运行的引擎
agent_executor = AgentExecutor(agent=main_agent, tools=tools, verbose=True)

In [45]:
# 执行指令
final_prompt = """
请分析我提供的数据，完成以下任务：
1. 检验'产品A_评分'和'产品B_评分'这两组数据的均值是否存在显著差异。
2. 在数轴上绘制点展示产品的评分，每个点代表一个评分，用不同颜色的点分别展示产品A_评分与产品B_评分。不要调用plt.show()展示图表。
3. 将生成的图表保存为'ratings_analysis.png'。
4. 最后，总结你所有的发现。
"""

# 运行工作流
response = agent_executor.invoke({"input": final_prompt})

print("\n工作流执行完毕，最终答案：")
print(response['output'])



> Entering new AgentExecutor chain...
 我需要依次完成四个任务：首先进行T检验比较两组评分的均值差异，然后绘制评分分布图并保存，最后总结发现。

Action: t_test_tool
Action Input: 产品A_评分,产品B_评分检验结果显著：T统计量为 3.66, P值为 0.0018。两组数据的均值存在显著差异。现在我知道两组评分存在显著差异，接下来需要绘制评分分布图。

Action: Pandas Agent
Action Input: 请用散点图展示'产品A_评分'和'产品B_评分'的分布，x轴为产品类型，y轴为评分值，用不同颜色区分产品A和B，不要调用plt.show()

> Entering new AgentExecutor chain...
Thought: 我需要使用matplotlib来创建散点图，x轴应该是产品类型（A或B），y轴是评分值。由于数据是两列分别代表产品A和B的评分，我需要先重组数据。

Action: python_repl_ast
Action Input: 
import matplotlib.pyplot as plt
import pandas as pd

# 重组数据为长格式
df_melted = df.melt(value_vars=['产品A_评分', '产品B_评分'], var_name='产品类型', value_name='评分')

# 创建散点图
plt.figure(figsize=(10, 6))
plt.scatter(x='产品类型', y='评分', data=df_melted, c=df_melted['产品类型'].map({'产品A_评分':'blue', '产品B_评分':'red'}), alpha=0.7)
plt.xlabel('产品类型')
plt.ylabel('评分值')
plt.title('产品A和B的评分分布')Text(0.5, 1.0, '产品A和B的评分分布')我成功创建了散点图，x轴显示产品类型，y轴显示评分值，并用不同颜色区分了产品A和B。

Final Answer: 散点图已创建，x轴为产品类型（A或B），y轴为评分值，产品A用蓝色表示，产品B用红色表示。图形标题为"产品A和B的评分分布"。

